# 03DK. Scenario analysis, Denmark-wide
## Project: Bicycle node network loop analysis

This notebook analyses loops created in notebook 01 with different scenarios for the whole country of Denmark.  
Explore two scenarios:  
1. Family with small children: 5-20km, max slope 4%
1. Teenage/adult leisure cyclist: 10-40km, max slope 6%

Contact: Michael Szell (michael.szell@gmail.com)

Created: 2024-10-25  
Last modified: 2025-08-01

## To do
- [X] Refactor with functions for restrictions
- [X] POI analysis
- [X] Fix all same max ylim values at ~8000

## To do future notebooks/viz

- [X] Polish nw plotting: colors, proj, transparency, keep values fixed
- [ ] Distribution/spatial clustering of slopes, pois in DK. How does it affect loops? Compare with random redistribution. Easy to add few POIS to dramatically increase coverage? "identify/fix POI deserts"
- [ ] Is node density a proxy for loop census size?

## Parameters

In [ ]:
%run -i setup_parameters.py
load_data = True  # Set to False if data are huge and have already been loaded
debug = True  # Set to True for extra plots and verbosity

In [ ]:
print("Running scenario in " + STUDY_AREA)
for k, v in SCENARIO[SCENARIOID].items():
    print(k + ": " + str(v))

## Functions

In [ ]:
%run -i functions.py

## Load data

This can take several minutes.

In [ ]:
if load_data:
    if LOOP_LENGTH_BOUND:
        llb_string = "_maxlength" + str(LOOP_LENGTH_BOUND)
    else:
        llb_string = ""

    with open(
        PATH["data_out"]
        + "loopcensus_"
        + str(LOOP_NUMNODE_BOUND)
        + llb_string
        + ".pkl",
        "rb",
    ) as f:
        allloops = pickle.load(f)
        alllooplengths = pickle.load(f)
        allloopnumnodes = pickle.load(f)
        allloopmaxslopes = pickle.load(f)
        Gnx = pickle.load(f)
        LOOP_NUMNODE_BOUND = pickle.load(f)
        nodes_id = pickle.load(f)
        nodes_coords = pickle.load(f)
        numloops = pickle.load(f)
        faceloops = pickle.load(f)

In [ ]:
# Create gdf and igraph versions
nodes, edges = momepy.nx_to_gdf(net=Gnx, points=True, lines=True)
G = ig.Graph.from_networkx(Gnx)
G.summary()

In [ ]:
# Plot network
if debug:
    plot_dk_gdf(
        nodes,
        edges,
        scale=0.4,
        vertex_size=get_vertex_size_constant(G.vcount()),
        link_width=get_edgewidth_constant(G.ecount()),
    )

In [ ]:
# Plot all loops
vertex_sizes, vertex_colors = get_vertex_plotinfo(allloops)
fig, ax = plot_dk_gdf(
    nodes,
    edges,
    scale=1,
    vertex_size=vertex_sizes,
    vertex_color=vertex_colors,
    link_width=0.3,
    link_color="#444444",
)
plot_dk_inset(fig, allloops, 8, 7800)
fig.savefig(PATH["plot"] + "network_numloops_all" + "." + PLOTPARAM["format"])

## Analyse

### Restrict to scenario lengths

In [ ]:
allloops_bound = {}
for sourcenode in tqdm(allloops, desc="Restrict to scenario lengths"):
    try:
        lengths_this = allloops[sourcenode]["lengths"] * MPERUNIT
        mask_this = (lengths_this >= SCENARIO[SCENARIOID]["looplength_min"]) & (
            lengths_this <= SCENARIO[SCENARIOID]["looplength_max"]
        )
        allloops_bound[sourcenode] = mask_node(allloops[sourcenode], mask_this)
    except:  # Account for 0 loop nodes
        allloops_bound[sourcenode] = {}

In [ ]:
# Plot all bounded loops
vertex_sizes, vertex_colors = get_vertex_plotinfo(allloops_bound)
fig, ax = plot_dk_gdf(
    nodes,
    edges,
    scale=1,
    vertex_size=vertex_sizes,
    vertex_color=vertex_colors,
    link_width=0.3,
    link_color=PLOTPARAM["color"]["neutral"],
)
plot_dk_scenariotext(ax, filterdepth=1)
plot_dk_inset(fig, allloops_bound, 8, 7800)

fig.savefig(
    PATH["plot"]
    + "network_numloops_"
    + str(SCENARIO[SCENARIOID]["looplength_min"])
    + "to"
    + str(SCENARIO[SCENARIOID]["looplength_max"])
    + "."
    + PLOTPARAM["format"]
)

### Restrict to scenario gradients

In [ ]:
allloops_bound_sloped = {}
for sourcenode in tqdm(allloops_bound, desc="Restrict to scenario gradients"):
    try:
        lengths_this = allloops[sourcenode]["lengths"] * MPERUNIT
        maxslopes_this = (
            allloops[sourcenode]["max_slopes"] / 100.0
        )  # max_slopes were multiplied by 100 for storage as uint16
        mask_this = lengths_this >= SCENARIO[SCENARIOID]["looplength_min"]
        mask_this &= lengths_this <= SCENARIO[SCENARIOID]["looplength_max"]
        mask_this &= maxslopes_this <= SCENARIO[SCENARIOID]["maxslope_limit"]
        allloops_bound_sloped[sourcenode] = mask_node(allloops[sourcenode], mask_this)
    except:  # Account for 0 loop nodes
        allloops_bound_sloped[sourcenode] = {}

In [ ]:
# Plot all bounded sloped loops
vertex_sizes, vertex_colors = get_vertex_plotinfo(allloops_bound_sloped)
link_widths, link_colors = get_link_plotinfo(
    edges,
    var_bad="max_slope",
    var_good="poi_diversity",
    threshold_bad=SCENARIO[SCENARIOID]["maxslope_limit"],
    threshold_good=0,
)
fig, ax = plot_dk_gdf(
    nodes,
    edges,
    scale=1,
    vertex_size=vertex_sizes,
    vertex_color=vertex_colors,
    link_width=link_widths,
    link_color=link_colors,
)
plot_dk_scenariotext(ax, filterdepth=2)
plot_dk_inset(fig, allloops_bound_sloped, 8, 7800)

fig.savefig(
    PATH["plot"]
    + "network_numloops_"
    + str(SCENARIO[SCENARIOID]["looplength_min"])
    + "to"
    + str(SCENARIO[SCENARIOID]["looplength_max"])
    + "_maxslope"
    + str(SCENARIO[SCENARIOID]["maxslope_limit"])
    + "."
    + PLOTPARAM["format"]
)

### Restrict to water limits

In [ ]:
allloops_bound_sloped_watered = {}
for sourcenode in tqdm(allloops_bound_sloped, desc="Restrict to water limits"):
    try:
        numloops = len(allloops_bound_sloped[sourcenode]["loops"])
        mask_this = [True] * numloops
        for i in range(numloops):
            wp = allloops_bound_sloped[sourcenode]["water_profile"][i]
            water_enough = True
            if wp:  # There is water on the way somewhere. Check distances
                for w in wp:
                    if w > WATERLENGTH_MAX:
                        water_enough = False
                        break
                if water_enough and (
                    allloops_bound_sloped[sourcenode]["lengths"][i] - wp[-1]
                    > WATERLENGTH_MAX
                ):
                    water_enough = False
            else:  # No water on the way, so the loop is only valid if short enough
                if allloops_bound_sloped[sourcenode]["lengths"][i] > WATERLENGTH_MAX:
                    water_enough = False
            mask_this[i] = water_enough
        allloops_bound_sloped_watered[sourcenode] = mask_node(
            allloops_bound_sloped[sourcenode], mask_this
        )
    except:  # Account for 0 loop nodes
        allloops_bound_sloped_watered[sourcenode] = {}

In [ ]:
# Plot all bounded sloped loops
vertex_sizes, vertex_colors = get_vertex_plotinfo(allloops_bound_sloped_watered)
link_widths, link_colors = get_link_plotinfo(
    edges,
    var_bad="max_slope",
    var_good="poi_diversity",
    threshold_bad=SCENARIO[SCENARIOID]["maxslope_limit"],
    threshold_good=1,
)
fig, ax = plot_dk_gdf(
    nodes,
    edges,
    scale=1,
    vertex_size=vertex_sizes,
    vertex_color=vertex_colors,
    link_width=link_widths,
    link_color=link_colors,
)
plot_dk_scenariotext(ax, filterdepth=3)
plot_dk_inset(fig, allloops_bound_sloped_watered, 8, 7800)


fig.savefig(
    PATH["plot"]
    + "network_numloops_"
    + str(SCENARIO[SCENARIOID]["looplength_min"])
    + "to"
    + str(SCENARIO[SCENARIOID]["looplength_max"])
    + "_maxslope"
    + str(SCENARIO[SCENARIOID]["maxslope_limit"])
    + "_waterlength_max"
    + str(WATERLENGTH_MAX)
    + "."
    + PLOTPARAM["format"]
)

### Restrict with POI diversity

In [ ]:
allloops_bound_sloped_watered_diverse = {}
for sourcenode in tqdm(
    allloops_bound_sloped_watered, desc="Restrict with POI diversity"
):
    try:
        numloops = len(allloops_bound_sloped_watered[sourcenode]["loops"])
        mask_this = [False] * numloops
        for i in range(numloops):
            poidiv = allloops_bound_sloped_watered[sourcenode]["poi_diversity"][i]
            if poidiv >= 3:
                mask_this[i] = True
        allloops_bound_sloped_watered_diverse[sourcenode] = mask_node(
            allloops_bound_sloped_watered[sourcenode], mask_this
        )
    except:  # Account for 0 loop nodes
        allloops_bound_sloped_watered_diverse[sourcenode] = {}

In [ ]:
# Plot all bounded sloped loops
vertex_sizes, vertex_colors = get_vertex_plotinfo(allloops_bound_sloped_watered_diverse)
link_widths, link_colors = get_link_plotinfo(
    edges,
    var_bad="max_slope",
    var_good="poi_diversity",
    threshold_bad=SCENARIO[SCENARIOID]["maxslope_limit"],
    threshold_good=3,
)
fig, ax = plot_dk_gdf(
    nodes,
    edges,
    scale=1,
    vertex_size=vertex_sizes,
    vertex_color=vertex_colors,
    link_width=link_widths,
    link_color=link_colors,
)
plot_dk_scenariotext(ax, filterdepth=4)
plot_dk_inset(fig, allloops_bound_sloped_watered_diverse, 8, 7800)

fig.savefig(
    PATH["plot"]
    + "network_numloops_"
    + str(SCENARIO[SCENARIOID]["looplength_min"])
    + "to"
    + str(SCENARIO[SCENARIOID]["looplength_max"])
    + "_maxslope"
    + str(SCENARIO[SCENARIOID]["maxslope_limit"])
    + "_waterlength_max"
    + str(WATERLENGTH_MAX)
    + "_poidiversity3"
    + "."
    + PLOTPARAM["format"]
)